## Librarires

In [67]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import pandas as pd


db logic

In [68]:
# embeddings = OllamaEmbeddings(model="mistral")

from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
# PDF file paths
# pdf_filepaths = [
#     "AVANA DRAFT-Draft.pdf",
#     "AVANA DRAFT-Reference.pdf"
# ]

# Load and process each PDF separately
# databases = []
# for filepath in pdf_filepaths:
#     # Load PDF and split into pages
#     loader = PyPDFLoader(filepath)  # Adjusted initialization without the keyword
#     pages = loader.load_and_split()
    
#     # Split pages into manageable chunks
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     document_splits = text_splitter.split_documents(documents=pages)
    
#     # Initialize and fill the database for each PDF
#     db = FAISS.from_documents(document_splits, embeddings)
#     databases.append(db)

# # Now you have a list of databases, each containing the vectorized content of one PDF
# print(databases)












pdf_filepaths = [
    "AVANA DRAFT-Draft.pdf",
    "AVANA DRAFT-Reference.pdf"
    # Add more file paths as needed
]
loaders = [PyPDFLoader(filepath) for filepath in pdf_filepaths]
# # Load and split the PDFs into pages
pages = [loader.load_and_split() for loader in loaders]
print(pages)
# Flatten the list of lists into a single list of pages
flat_pages = [page for sublist in pages for page in sublist]

# Define the text splitter for manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents=flat_pages)

# Initialize the Chroma vector store and create it using the document splits
db = FAISS.from_documents(all_splits, embeddings)


c:\Users\DELL\miniconda3\envs\shipment\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[Document(page_content='DRAFT BILL OF LADING NOT NEGOTIABLE UNLESS CONSIGNED "TO ORDER"\nUnifeeder FZCO\nPUNJAB RICELAND AGRO FOODS PRIVATE LIMITED.\nPLOT NO. 324,NEW GIDC, MITHIROHAR  NEAR  MET INDIA \nGANDHIDHAM,GUJARAT  PIN-370201MUN/JEA/24/14740\nShipper\'s Ref. F/Agent Name & Ref.Shipper Country of OriginBill of Lading No.\nRUSHABH SEALINK AND \nLOGISTIC PRIVATE LIMITEDINDIA\nConsignee (if "To Order" so indicate)\nAL JANAN GENERAL TRADING CO. LLC\nAI KHAN ROAD, INDUSTRIAL AREA 1 SHARJAH, UAE - PO BOX NO. 38854 \nT: +97165331498 E: ALJANAN.SHPI@YAHOO.CO.UKNotify2\nNotify1(No claim shall attach for the failure to notify) Agent\nTHE BUSINESS CENTRE BUILDING, 6TH FLOOR, UNIT 604, \nKHALID BIN WALEED STREET, BUR DUBAI, DUBAI, U.A.E.\nTEL : 04 3530258 FAX : 04 3530248 TOLL FREE : 8000321344 \n EMAIL ID MYLENE.AMABA@UNIFEEDER.COM, \nTHUSITHA.PRIYANKARA@UNIFEEDER.COM, DEEPAK.S@UNIFEEDER.COM., \nIMP.TSL@UNIFEEDER.COM\nPhone:04 3530258 AI KHAN ROAD, INDUSTRIAL AREA 1 SHARJAH, UAE - PO BOX 

In [66]:
# Based on the provided context, do side-by-side comparison in tabular formate like below pandas DataFrame Formate:

from langchain_core.prompts import ChatPromptTemplate
template = """
Please compare all the features from two documents and fill in the table:


{context}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = Ollama(model="mistral")



document_chain = create_stuff_documents_chain(llm,prompt)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
# Assuming 'invoke' needs to pass a dictionary with specific keys
# response = retrieval_chain.invoke({"context": "extract the necessary values for comparison from the documents"})

response = retrieval_chain.invoke({"input": "compare both document side by side and give output in json formate "})
print(response)
try:
    comparison_data = [
        {"Feature": "BL Number", "Document 1": response.get('BL Number', [None, None])[0], "Document 2": response.get('BL Number', [None, None])[1]},
        {"Feature": "Vessel Name", "Document 1": response.get('Vessel Name', [None, None])[0], "Document 2": response.get('Vessel Name', [None, None])[1]}
        # Add more features based on available data
    ]

    # Create the DataFrame
    df = pd.DataFrame(comparison_data)
    print(df)
except KeyError as e:
    print("KeyError encountered:", e)
    print("Check the keys in the response and adjust the code accordingly.")

{'input': 'compare both document side by side and give output in json formate ', 'context': [Document(page_content=': BL NumberVessel Name Voyage\nMUN/JEA/24/14740 SC MARA 24015WNOT NEGOTIABLE UNLESS CONSIGNED "TO ORDER"\nBL ATTACHMENT\nNumber and kind of packages,\nDescription of goods.Marks and NumberContainer No. \nSeal No.\nNET WEIGHT (TOTAL) : 25.004 MT\nGROSS WEIGHT (TOTAL) : 25.142 MT\nPACKING DATE : 03/2024   EXPIRY DATE : 02/2026\nPACKING MARK: "RAHMANI"\nIEC CODE : 0315903040\nHS CODE:10063020\nBUYER\'S ORDER & DATE \nHVGTF/2024-25/1081    DATED: 09-03-2024\nINVOICE NO. PRL/GDM/EXP/484 DATE: 09.04.2024\nS/BILL NO:  DATE: 09.04.2024\nNOTIFY PARTY 2 : \nHARMONY VENTURES GENERAL TRADING FZCO\nADDRESS: FZJOAB0909 JEBEL ALI FREEZONE\nUNITED ARAB EMIRATES.\nHARMONYVGT@GMAIL.COM\nCONTACT NUMBER : 0523848350', metadata={'source': 'AVANA DRAFT-Draft.pdf', 'page': 1}), Document(page_content='Description of Goods\n01X20\'FCL TOTAL : 658 BAGS GROSS WT\n658 Bags of Indian Basmati Rice Pac